# Ceneo Scraper

## Loading libraries

In [ ]:
import requests
from bs4 import BeautifulSoup

## Sending requests to Ceneo.pl server

In [ ]:
product_id = "158793489"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)

In [ ]:
page_dom= BeautifulSoup(response.text, "html.parser")
# print(page_dom)
opinions = page_dom.select("div.js_product-review")
opinion = page_dom.select_one("div.js_product-review")
print(type(page_dom))
print(type(opinions))
print(type(opinion))
print(opinion)


## Extraction of components of single opinion

|component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|user-post__author-name|author|
|author’s recommendation|user-post__author-recomendation >  em|recommendation|
|score expressed in number of stars|user-post__score-count|score|
|opinion’s content|div.user-post__text|content|
|list of product advantages|div.review-feature__title review-feature__title--positives ~ div.review-feature__item|pros|
|list of product disadvantages|div.review-feature__title review-feature__title--negatives ~ div.review-feature__item|cons|
|how many users think that opinion was helpful|button.votes-yes > span|helpful|
|how many users think that opinion was unhelpful|button.votes-no > span|unhelpful|
|publishing date|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|purchase date|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|



In [ ]:
opinion_id = opinion["data-entry-id"]
author = opinion.select_one(["user-post__author-name"])
recommendation = [""]